In this notebook, the most important metrics from the forecast results are calculated. Those values were then used in the final report.

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_pinball_loss
import os
from pathlib import Path

# Go one directory up, to get a parent directory with utils
par_dir = os.path.dirname(os.getcwd())
os.chdir(par_dir)

from utils import ReadAllData
from feature_engineering import preprocess_snotel
from cv_and_hyperparams_opt import interval_coverage

2024-04-10 19:58:35.257 | INFO     | wsfr_read.config:<module>:10 - DATA_ROOT is C:\Users\ja1\Documents\water_supply_forecast_rodeo_competition\data
C:\Users\ja1\anaconda3\envs\water_supply_hindcast_test\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Prepare data

In [2]:
def get_final_results(results_df: pd.DataFrame) -> tuple[float, float]:
    """
    Given predictions from the given DataFrame, calculate average values for each
    quantile, final results and final interval coverage. Print those values.
    
    Args:
        results_df (pd.DataFrame): a DataFrame with results (predictions)
    Returns:
        metric (float): Competition metric
        interval (float): Interval coverage average
    """
    result_10 = mean_pinball_loss(results_df.volume,
                                  results_df.volume_10,
                                  alpha = 0.1)
    result_50 = mean_pinball_loss(results_df.volume,
                                  results_df.volume_50,
                                  alpha = 0.5)
    result_90 = mean_pinball_loss(results_df.volume,
                                  results_df.volume_90,
                                  alpha = 0.9)
    print(f'Results from different quantiles: {result_10}, {result_50}, {result_90}')
    metric = 2 * (result_10 + result_50 + result_90) / 3
    print(f'Final results: {metric}')
    interval = interval_coverage(results_df.volume,
                                 np.array(results_df[['volume_10', 'volume_50', 'volume_90']]))
    print(f'Final interval coverage: {interval}')
    return metric, interval

In [3]:
# Read predictions and labels
labels = pd.read_csv('data/cross_validation_labels.csv')
preds = pd.read_pickle('results/submission_2024_03_28.pkl')

In [4]:
# Create graphs\ directory if it doesn't exist
graphs_path = Path('graphs')
graphs_path.mkdir(parents = True, exist_ok = True)

# General results

In [5]:
# Merge predictions with labels
preds['year'] = preds.issue_date.str[:4].astype('int')
results = pd.merge(preds,
                   labels,
                   how = 'left',
                   on = ['site_id', 'year'])
preds['year'] = preds.year.astype('str')

In [6]:
# Get final results
metric, interval = get_final_results(results)

Results from different quantiles: 27.040071062422165, 67.03886111491363, 36.092474972603725
Final results: 86.78093809995967
Final interval coverage: 0.7897099447513812


## General results without outlier year 2011

In [7]:
metric, interval = get_final_results(results[results.year != 2011])

Results from different quantiles: 25.62011098948795, 64.48390569998554, 33.76894973791258
Final results: 82.58197761825738
Final interval coverage: 0.8027042744984006


# By issue month

In [8]:
# Get month from issue dates
results['month'] = results.issue_date.str[5:7].astype('int')

# Get results for each issue month separately
months = [1, 2, 3, 4, 5, 6, 7]
metric_monthly = []
interval_monthly = []
for month in months:
    metric, interval = get_final_results(results[results.month == month])
    # Apend results and interval coverage from each month
    metric_monthly.append(metric)
    interval_monthly.append(interval)

Results from different quantiles: 39.88731107932785, 108.05436870956517, 56.81127473446953
Final results: 136.5019696822417
Final interval coverage: 0.7721153846153846
Results from different quantiles: 37.75591566088068, 96.60182597061791, 49.470080796770034
Final results: 122.55188161884575
Final interval coverage: 0.7725961538461539
Results from different quantiles: 34.73936498098114, 85.9528622410915, 46.27961821038274
Final results: 111.31456362163692
Final interval coverage: 0.7836538461538461
Results from different quantiles: 28.64418177843136, 67.4148779422614, 36.0699917574015
Final results: 88.0860343187295
Final interval coverage: 0.79375
Results from different quantiles: 22.75321148418274, 51.18266431188423, 29.53642779691043
Final results: 68.9815357286516
Final interval coverage: 0.7990384615384616
Results from different quantiles: 15.83468343038915, 38.51060882725041, 22.55897401831103
Final results: 51.26951085063373
Final interval coverage: 0.7961538461538461
Results fr

## Create a DataFrame with results

In [9]:
results_monthly = pd.DataFrame(data = (months, metric_monthly, interval_monthly)).T
results_monthly.columns = ['Month', 'MQL Result', 'Interval coverage']
results_monthly['Month'] = results_monthly.Month.astype('int')

results_monthly

,Month,MQL Result,Interval coverage
0,1,136.501970,0.772115
1,2,122.551882,0.772596
2,3,111.314564,0.783654
3,4,88.086034,0.793750
4,5,68.981536,0.799038
5,6,51.269511,0.796154
6,7,26.440276,0.811500


In [10]:
# Save results to Excel
# results_monthly.to_excel('graphs/results_monthly.xlsx', index = False)

# By Year

## Get yearly results

In [11]:
years = results.year.unique()
metric_yearly = []
interval_yearly = []

for year in years:
    print(year)
    metric, interval = get_final_results(results[results.year == year])
    metric_yearly.append(metric)
    interval_yearly.append(interval)

2004
Results from different quantiles: 13.798106047058862, 62.59079762542434, 34.707791394270615
Final results: 74.06446337783588
Final interval coverage: 0.8425414364640884
2005
Results from different quantiles: 34.95586088438077, 70.01993006686452, 29.46873391892375
Final results: 89.62968324677935
Final interval coverage: 0.819060773480663
2006
Results from different quantiles: 30.190207356365235, 73.82968035840364, 30.568033489412347
Final results: 89.72528080278748
Final interval coverage: 0.7997237569060773
2007
Results from different quantiles: 14.465633593685107, 50.41134301983809, 34.69797147067942
Final results: 66.38329872280174
Final interval coverage: 0.8701657458563536
2008
Results from different quantiles: 23.901630141875106, 64.84324331968492, 30.428826876481132
Final results: 79.4491335586941
Final interval coverage: 0.8701657458563536
2009
Results from different quantiles: 26.266108902849666, 64.28512009846914, 23.490143858407492
Final results: 76.0275819064842
Final 

## Create a DataFrame with results

In [12]:
results_yearly = pd.DataFrame(data = (years, metric_yearly, interval_yearly)).T
results_yearly.columns = ['Year', 'Results', 'Interval coverage']
results_yearly['Year'] = results_yearly.Year.astype('int')

results_yearly

,Year,Results,Interval coverage
0,2004,74.064463,0.842541
1,2005,89.629683,0.819061
2,2006,89.725281,0.799724
3,2007,66.383299,0.870166
4,2008,79.449134,0.870166
5,2009,76.027582,0.870166
6,2010,79.973587,0.897790
7,2011,166.561187,0.542818
8,2012,127.513054,0.694751
9,2013,65.065328,0.845304


## Append auxiliary columns
Those features will be used in the final report to look for patterns in results from different periods.

### Add volume per site_id
Sum of volumes from different site ids per water year

In [13]:
volume_sum_per_year = labels.groupby('year')['volume'].sum()
results_yearly = pd.merge(results_yearly,
                          volume_sum_per_year,
                          how = 'left',
                          left_on = 'Year',
                          right_index = True)
results_yearly['volume'] = results_yearly.volume.astype('int')
results_yearly.rename({'volume': 'Volume'}, axis = 1, inplace = True)

results_yearly

,Year,Results,Interval coverage,Volume
0,2004,74.064463,0.842541,16579
1,2005,89.629683,0.819061,22537
2,2006,89.725281,0.799724,28335
3,2007,66.383299,0.870166,18036
4,2008,79.449134,0.870166,24405
5,2009,76.027582,0.870166,23015
6,2010,79.973587,0.897790,21715
7,2011,166.561187,0.542818,37294
8,2012,127.513054,0.694751,24813
9,2013,65.065328,0.845304,18908


### Add SNOTEL Precipitation
Sum of maximum values of precipitation from different site ids per year. The maximum value of precipitation per water year (precipitation in SNOTEL is aggregated, so it should be the value from July 21)

In [14]:
# Get SNOTEL data
dfs = ReadAllData()
snotel = preprocess_snotel(dfs.snotel,
                           dfs.sites_to_snotel_stations)

# Get sum of values from each site id per year
prec_max_yearly_sum_sites = snotel.groupby(['year_forecast', 'site_id'])['PREC_DAILY'].max().reset_index().\
    groupby('year_forecast')['PREC_DAILY'].sum()
# Merge with yearly results
results_yearly = pd.merge(results_yearly,
                          prec_max_yearly_sum_sites,
                          how = 'left',
                          left_on = 'Year',
                          right_index = True)
# Change names of SNOTEL columns
results_yearly.rename({'PREC_DAILY': 'SNOTEL Precipitation'},
                      axis = 1, inplace = True)
results_yearly

,Year,Results,Interval coverage,Volume,SNOTEL Precipitation
0,2004,74.064463,0.842541,16579,831.476293
1,2005,89.629683,0.819061,22537,889.664871
2,2006,89.725281,0.799724,28335,960.231134
3,2007,66.383299,0.870166,18036,866.786391
4,2008,79.449134,0.870166,24405,956.906523
5,2009,76.027582,0.870166,23015,923.326465
6,2010,79.973587,0.897790,21715,907.599951
7,2011,166.561187,0.542818,37294,1201.410260
8,2012,127.513054,0.694751,24813,903.271770
9,2013,65.065328,0.845304,18908,857.896233


In [15]:
# Save results to Excel
# results_yearly.to_excel('graphs/results_yearly_snotel.xlsx', index = False)

# Per site id

In [16]:
site_ids = results.site_id.unique()
metric_site_ids = []
interval_site_ids = []

for site_id in site_ids:
    print(site_id)
    metric, interval = get_final_results(results[results.site_id == site_id])
    metric_site_ids.append(metric)
    interval_site_ids.append(interval)

hungry_horse_reservoir_inflow
Results from different quantiles: 47.60991207604558, 122.12525249670693, 60.4979940798842
Final results: 153.48877243509114
Final interval coverage: 0.7839285714285714
snake_r_nr_heise
Results from different quantiles: 73.79308080416622, 166.1239057522856, 80.26690030703082
Final results: 213.4559245756551
Final interval coverage: 0.8267857142857142
pueblo_reservoir_inflow
Results from different quantiles: 11.600697501259189, 30.28702681261669, 21.299879113576164
Final results: 42.125068951634695
Final interval coverage: 0.8696428571428572
sweetwater_r_nr_alcova
Results from different quantiles: 2.5397471254024837, 8.21530234478722, 4.694532573514508
Final results: 10.299721362469475
Final interval coverage: 0.7392857142857143
missouri_r_at_toston
Results from different quantiles: 58.0398731208155, 149.69737554852153, 85.19186488547354
Final results: 195.28607570320705
Final interval coverage: 0.8321428571428572
animas_r_at_durango
Results from different q

## Create a DataFrame with results

In [17]:
results_site_ids = pd.DataFrame(data = (site_ids, metric_site_ids, interval_site_ids)).T
results_site_ids.columns = ['Site id', 'Results', 'Interval coverage']

## Append auxiliary columns
Those features will be used in the final report to look for patterns in results from different site ids.

### Add volume per site_id
Average volume per site id

In [18]:
#Append averaae volume per site id
site_id_avg_per_year = labels.groupby('site_id')['volume'].mean()
results_site_ids = pd.merge(results_site_ids,
                          site_id_avg_per_year,
                          how = 'left',
                          left_on = 'Site id',
                          right_index = True)
results_site_ids['volume'] = results_site_ids.volume.astype('int')
results_site_ids.rename({'volume': 'Avg volume'}, axis = 1, inplace = True)

### Add Ratio MQL to Avg volume
Results are strongly correlated with the average Water flow volume. To better assess predictive power of the given site id, it seems reasonable to exclude average volume impact.

In [19]:
results_site_ids['Ratio MQL to Avg volume'] = results_site_ids['Results'] / results_site_ids['Avg volume']

## Add SNOTEL avg Snow Water Equivalent
Average Snow Water Equivalent per site id

In [20]:
snotel_avg_site_id = snotel.groupby(['site_id'])['WTEQ_DAILY'].mean()

results_site_ids = pd.merge(results_site_ids,
                          snotel_avg_site_id,
                          how = 'left',
                          left_on = 'Site id',
                          right_index = True)

results_site_ids.rename({'WTEQ_DAILY': 'SNOTEL avg Snow Water Equivalent'}, axis = 1, inplace = True)

# Sort values by volume
results_site_ids = results_site_ids.sort_values('Avg volume').reset_index(drop = True)
results_site_ids

,Site id,Results,Interval coverage,Avg volume,Ratio MQL to Avg volume,SNOTEL avg Snow Water Equivalent
0,pecos_r_nr_pecos,6.53686,0.792857,44,0.148565,3.467899
1,sweetwater_r_nr_alcova,10.299721,0.739286,53,0.194334,5.649905
2,virgin_r_at_virtin,9.151598,0.830357,58,0.157786,4.906232
3,taylor_park_reservoir_inflow,9.494951,0.766071,89,0.106685,6.192263
4,weber_r_nr_oakley,11.29966,0.798214,107,0.105604,7.406301
5,colville_r_at_kettle_falls,19.959118,0.735714,123,0.162269,13.910979
6,ruedi_reservoir_inflow,12.476969,0.773214,129,0.096721,6.865278
7,dillon_reservoir_inflow,15.76545,0.7625,156,0.101061,6.287008
8,green_r_bl_howard_a_hanson_dam,25.415136,0.826786,249,0.102069,14.767083
9,owyhee_r_bl_owyhee_dam,68.271053,0.766071,315,0.216734,5.287738


In [21]:
# Save results to Excel
# results_site_ids.to_excel('graphs/results_site_ids.xlsx', index = False)

# Bonus prizes
Some site ids/predictive periods were of special importance in the competition (https://www.drivendata.org/competitions/262/reclamation-water-supply-forecast-final/page/873/#subcategory-bonus-prizes). Below are presented results from those special features.

In [22]:
# Initialize a list with bonus results
bonus_results = []

## By site ids

In [23]:
cascades = ['skagit_ross_reservoir',
            'stehekin_r_at_stehekin',
            'green_r_bl_howard_a_hanson_dam',
            'detroit_lake_inflow']

metric, interval = get_final_results(results[results.site_id.isin(cascades)])

bonus_results.append(['Cascades', ', '.join(cascades), metric, interval])

Results from different quantiles: 21.379032433669998, 44.31838323659638, 20.750467339252282
Final results: 57.631922006345775
Final interval coverage: 0.7518518518518519


In [24]:
sierra_nevada = ['san_joaquin_river_millerton_reservoir',
                 'merced_river_yosemite_at_pohono_bridge',
                 'american_river_folsom_lake']

metric, interval = get_final_results(results[results.site_id.isin(sierra_nevada)])

bonus_results.append(['Sierra Nevada', ', '.join(sierra_nevada), metric, interval])

Results from different quantiles: 45.94118220809306, 109.12618988379039, 55.74399719549294
Final results: 140.54091285825092
Final interval coverage: 0.7696428571428572


In [25]:
colorado_headwaters = ['ruedi_reservoir_inflow',
                       'dillon_reservoir_inflow',
                       'taylor_park_reservoir_inflow',
                       'animas_r_at_durango']

metric, interval = get_final_results(results[results.site_id.isin(colorado_headwaters)])

bonus_results.append(['Colorado Headwaters', ', '.join(colorado_headwaters), metric, interval])

Results from different quantiles: 5.112038073465333, 14.636792722593334, 8.1660613656661
Final results: 18.60992810781651
Final interval coverage: 0.7959821428571429


In [26]:
challenging_basins = ['owyhee_r_bl_owyhee_dam',
                      'virgin_r_at_virtin',
                      'pecos_r_nr_pecos']

metric, interval = get_final_results(results[results.site_id.isin(challenging_basins)])

bonus_results.append(['Chalenging basins', ', '.join(challenging_basins), metric, interval])

Results from different quantiles: 6.403026515714653, 20.746050301182187, 14.830678291611006
Final results: 27.986503405671897
Final interval coverage: 0.7964285714285714


## By Early Lead Time Bonus Prize
Issue date from January 1 through March 15

In [27]:
# Get day
results['day'] = results.issue_date.str[-2:].astype('int')
# Select data up to March 15
results_early_lead_time = results[(results.month <= 2) | ((results.month == 3) & (results.day <= 15))]
# Get results
metric, interval = get_final_results(results_early_lead_time)
bonus_results.append(['Early lead time', 'Issue dates up to Mar 15', metric, interval])

Results from different quantiles: 37.85309996612358, 98.36237070932978, 51.39332501014964
Final results: 125.07253045706868
Final interval coverage: 0.7744755244755245


## Create a DataFrame with results

In [28]:
# Transform bonus results to a DataFrame
bonus_results = pd.DataFrame(bonus_results)
bonus_results.columns = ['Bonus subcategory', 'Subcategory details', 'MQL Result', 'Interval coverage']
bonus_results

,Bonus subcategory,Subcategory details,MQL Result,Interval coverage
0,Cascades,"skagit_ross_reservoir, stehekin_r_at_stehekin,...",57.631922,0.751852
1,Sierra Nevada,"san_joaquin_river_millerton_reservoir, merced_...",140.540913,0.769643
2,Colorado Headwaters,"ruedi_reservoir_inflow, dillon_reservoir_inflo...",18.609928,0.795982
3,Chalenging basins,"owyhee_r_bl_owyhee_dam, virgin_r_at_virtin, pe...",27.986503,0.796429
4,Early lead time,Issue dates up to Mar 15,125.072530,0.774476


In [29]:
# Save results to Excel
# bonus_results.to_excel('graphs/bonus_results.xlsx', index = False)